In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/KTH-TIPS2-b.zip .

In [4]:
import matplotlib.pyplot as plt
from keras.layers import (
    Input, Dense, Conv2D, Flatten, Reshape, Dropout, 
    Activation, AveragePooling2D, BatchNormalization,
    Lambda, Concatenate
)
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np
from gc import collect

In [5]:
def WaveletTransform(img):
    low = img[:,::2,...] + img[:,1::2,...]
    low = (low[:,:,::2,...] + low[:,:,1::2,...]) / 4
    diag = img[:,::2,...] - img[:,1::2,...]
    diag = diag[:,:,::2,...] - diag[:,:,1::2,...]
    top = img[:,::2,...] - img[:,1::2,...]
    top = (top[:,:,::2] + top[:,:,1::2] ) / 2
    left = img[:,:,::2,...] - img[:,:,1::2,...]
    left = (left[:,::2] + left[:,1::2] ) / 2
    
    return low, diag, top, left


def conv_layer(_in, N):
    conv1 = Conv2D(N, kernel_size=(3,3), padding='same')(_in)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)
    conv2 = Conv2D(N, kernel_size=(3,3), strides=(2,2), padding='same')(relu1)
    norm2 = BatchNormalization()(conv2)
    relu2 = Activation('relu')(norm2)
    return relu2


def build_model(input_shape = (256, 256, 3), num_classes=50):
    _input = BatchNormalization()(Input(input_shape))
    
    low1, diag1, top1, left1 = Lambda(WaveletTransform, name='wavelet_1')(_input)
    low2, diag2, top2, left2 = Lambda(WaveletTransform, name='wavelet_2')(low1)
    low3, diag3, top3, left3 = Lambda(WaveletTransform, name='wavelet_3')(low2)
    low4, diag4, top4, left4 = Lambda(WaveletTransform, name='wavelet_4')(low3)
    
    k1 = Concatenate()([conv_layer(_input,64), low1, diag1, top1, left1])
    k2 = Concatenate()([conv_layer(k1, 128), low2, diag2, top2, left2])
    k3 = Concatenate()([conv_layer(k2, 256), low3, diag3, top3, left3])
    k4 = Concatenate()([conv_layer(k3, 512), low4, diag4, top4, left4])
    
    avg_pool = AveragePooling2D(pool_size=(7,7), strides=1, padding='same')(k4)
    flat = Flatten()(avg_pool)
    output = Dense(num_classes, activation='softmax',name='fc')(flat)
    model = Model(inputs=_input, outputs=output)
    return model

In [6]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8133510189290938112
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14417788928
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9626492363071632862
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
2.9.2
Num GPUs Available:  1
True


In [7]:
full_dataset = image_dataset_from_directory('./KTH-TIPS2-b', label_mode='categorical')
print(full_dataset.class_names)
DATASET_SIZE = 4752
train_size = int(0.7 * DATASET_SIZE / 32)
val_size = int(0.15 * DATASET_SIZE / 32)
test_size = int(0.15 * DATASET_SIZE / 32)

full_dataset = full_dataset.shuffle(10_000)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)
# del full_dataset
n = next(train_dataset.as_numpy_iterator())
shape, num_classes = n[0].shape[1:], n[1].shape[-1]
model = build_model(input_shape=shape, num_classes=num_classes)

print(model.summary())

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('./checkpoints/', monitor='accuracy', save_best_only=True)
model.fit(train_dataset, validation_data=val_dataset, epochs=75, callbacks=[checkpoint])
model = load_model('./checkpoints/')
score = model.evaluate(test_dataset)
N = sum([_[1].shape[1] for _ in test_dataset.as_numpy_iterator()])
print(score)
print(1/np.sqrt(N))

Found 4752 files belonging to 11 classes.
['aluminium_foil', 'brown_bread', 'corduroy', 'cork', 'cotton', 'cracker', 'lettuce_leaf', 'linen', 'white_bread', 'wood', 'wool']
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization

103/103 [==============================] - 87s 635ms/step - loss: 49.7508 - accuracy: 0.3334 - val_loss: 3132.3005 - val_accuracy: 0.0911
Epoch 2/75
103/103 [==============================] - ETA: 0s - loss: 19.6010 - accuracy: 0.4809

103/103 [==============================] - 75s 650ms/step - loss: 19.6010 - accuracy: 0.4809 - val_loss: 342.2049 - val_accuracy: 0.2773
Epoch 3/75
103/103 [==============================] - ETA: 0s - loss: 15.2309 - accuracy: 0.6082

103/103 [==============================] - 75s 658ms/step - loss: 15.2309 - accuracy: 0.6082 - val_loss: 33.8423 - val_accuracy: 0.4727
Epoch 4/75
103/103 [==============================] - ETA: 0s - loss: 12.3387 - accuracy: 0.6890

103/103 [==============================] - 74s 649ms/step - loss: 12.3387 - accuracy: 0.6890 - val_loss: 51.9968 - val_accuracy: 0.4609
Epoch 5/75
103/103 [==============================] - ETA: 0s - loss: 9.8827 - accuracy: 0.7710

...

103/103 [==============================] - 74s 626ms/step - loss: 4.4423 - accuracy: 0.9707 - val_loss: 7.6068 - val_accuracy: 0.9219
Epoch 58/75
103/103 [==============================] - ETA: 0s - loss: 2.2459 - accuracy: 0.9718

103/103 [==============================] - 72s 627ms/step - loss: 2.2459 - accuracy: 0.9718 - val_loss: 63.7663 - val_accuracy: 0.7819
Epoch 59/75
103/103 [==============================] - 67s 575ms/step - loss: 3.0199 - accuracy: 0.9710 - val_loss: 50.5519 - val_accuracy: 0.7930
Epoch 60/75
103/103 [==============================] - ETA: 0s - loss: 1.8246 - accuracy: 0.9780

103/103 [==============================] - 72s 620ms/step - loss: 1.8246 - accuracy: 0.9780 - val_loss: 51.4457 - val_accuracy: 0.8242
Epoch 61/75
103/103 [==============================] - 67s 576ms/step - loss: 3.6595 - accuracy: 0.9649 - val_loss: 303.0944 - val_accuracy: 0.5521
Epoch 62/75
103/103 [==============================] - 67s 575ms/step - loss: 3.0489 - accuracy: 0.9739 - val_loss: 49.4747 - val_accuracy: 0.7695
Epoch 63/75
103/103 [==============================] - 67s 573ms/step - loss: 2.1369 - accuracy: 0.9759 - val_loss: 5.8696 - val_accuracy: 0.9375
Epoch 64/75
103/103 [==============================] - 67s 578ms/step - loss: 4.1644 - accuracy: 0.9640 - val_loss: 12.7470 - val_accuracy: 0.9271
Epoch 65/75
103/103 [==============================] - 67s 581ms/step - loss: 3.1791 - accuracy: 0.9709 - val_loss: 56.2026 - val_accuracy: 0.7695
Epoch 66/75
103/103 [==============================] - 67s 577ms/step - loss: 3.3201 - accuracy: 0.9692 - val_loss: 10.3370 - val_

103/103 [==============================] - 73s 619ms/step - loss: 2.3559 - accuracy: 0.9790 - val_loss: 51.7454 - val_accuracy: 0.8333
Epoch 68/75
103/103 [==============================] - ETA: 0s - loss: 2.3680 - accuracy: 0.9811

103/103 [==============================] - 71s 614ms/step - loss: 2.3680 - accuracy: 0.9811 - val_loss: 16.2369 - val_accuracy: 0.9245
Epoch 69/75
103/103 [==============================] - 67s 573ms/step - loss: 2.5044 - accuracy: 0.9794 - val_loss: 303.2146 - val_accuracy: 0.6159
Epoch 70/75
103/103 [==============================] - 67s 576ms/step - loss: 3.5994 - accuracy: 0.9692 - val_loss: 163.7932 - val_accuracy: 0.6719
Epoch 71/75
103/103 [==============================] - 67s 574ms/step - loss: 3.1230 - accuracy: 0.9741 - val_loss: 62.3200 - val_accuracy: 0.8320
Epoch 72/75
103/103 [==============================] - 66s 571ms/step - loss: 1.9985 - accuracy: 0.9771 - val_loss: 39.7948 - val_accuracy: 0.8164
Epoch 73/75
103/103 [==============================] - 67s 575ms/step - loss: 4.8100 - accuracy: 0.9646 - val_loss: 50.7820 - val_accuracy: 0.8255
Epoch 74/75
103/103 [==============================] - 66s 572ms/step - loss: 2.2048 - accuracy: 0.9771 - val_loss: 20.7045 - va